# Correlated (multivariate) normal random variables

In [1]:
import numpy as np

In [2]:
# Sigma (std) and correlation 
sig_v = np.array([2, 5, 2])
cor_m = np.array([[1, 0.7, -0.2], [0.7, 1, 0.5], [-0.2, 0.5, 1]])
print(sig_v, '\n', cor_m)

[2 5 2] 
 [[ 1.   0.7 -0.2]
 [ 0.7  1.   0.5]
 [-0.2  0.5  1. ]]


### Construct Covariance Matrix

In [3]:
sig_v[:,None]

array([[2],
       [5],
       [2]])

In [4]:
cov_m = sig_v * cor_m * sig_v[:,None]
print(cov_m)

[[ 4.   7.  -0.8]
 [ 7.  25.   5. ]
 [-0.8  5.   4. ]]


### Cholesky decomposition of covariance matrix

In [5]:
chol_m = np.linalg.cholesky(cov_m)
print(chol_m)

[[ 2.          0.          0.        ]
 [ 3.5         3.57071421  0.        ]
 [-0.4         1.79235851  0.79211803]]


In [6]:
# Let's verify that L x L^T = Covariance

print( chol_m @ chol_m.transpose(), '\n' )
print( chol_m @ chol_m.transpose() - cov_m )

[[ 4.   7.  -0.8]
 [ 7.  25.   5. ]
 [-0.8  5.   4. ]] 

[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]


In [7]:
# Now let's create multivariate normal random variables following the covariance matrix
# First, create standard normals (3 x 1000)

znorm_m = np.random.normal(size=(3, 10000))
znorm_m

array([[-0.71840949, -0.32858861,  0.17086077, ..., -0.75634578,
         0.30744772,  0.9555333 ],
       [-0.15495152,  0.45113022,  0.63607913, ..., -0.44831145,
         0.91787615,  0.40840932],
       [ 0.75153094, -1.43435177, -0.46546293, ...,  0.62524413,
         1.54544981,  0.41868011]])

In [8]:
np.round(np.cov(znorm_m),3)

array([[ 1.009, -0.02 ,  0.003],
       [-0.02 ,  0.998, -0.015],
       [ 0.003, -0.015,  1.02 ]])

In [9]:
# Then multiply C^T

#xnorm_m = znorm_m @ chol_m.transpose()
xnorm_m = chol_m @ znorm_m
xnorm_m

array([[-1.43681898, -0.65717722,  0.34172155, ..., -1.51269157,
         0.61489544,  1.9110666 ],
       [-3.06772081,  0.46079698,  2.8692695 , ..., -4.24800233,
         4.35354043,  4.80267952],
       [ 0.60493634, -0.19615336,  0.70303595, ..., -0.00572939,
         2.7463627 ,  0.68144666]])

In [10]:
# Let's verify that X = C * Z  follows the covariance
print(cov_m)
cov_m_sample = np.cov( xnorm_m )
print( 'Cov from sample:\n', cov_m_sample )
print( 'Error of Cov matrix:\n', cov_m_sample - cov_m )

[[ 4.   7.  -0.8]
 [ 7.  25.   5. ]
 [-0.8  5.   4. ]]
Cov from sample:
 [[ 4.03545891  6.91725387 -0.87479964]
 [ 6.91725387 24.57754141  4.8436276 ]
 [-0.87479964  4.8436276   3.99268149]]
Error of Cov matrix:
 [[ 0.03545891 -0.08274613 -0.07479964]
 [-0.08274613 -0.42245859 -0.1563724 ]
 [-0.07479964 -0.1563724  -0.00731851]]


In [11]:
# also check the correation
print(cor_m)
cor_m_sample = np.corrcoef( xnorm_m )
print( 'Corr from sample:\n', cor_m_sample )
print( 'Error:\n', cor_m_sample - cor_m )

[[ 1.   0.7 -0.2]
 [ 0.7  1.   0.5]
 [-0.2  0.5  1. ]]
Corr from sample:
 [[ 1.          0.69457322 -0.21793641]
 [ 0.69457322  1.          0.48895534]
 [-0.21793641  0.48895534  1.        ]]
Error:
 [[ 0.         -0.00542678 -0.01793641]
 [-0.00542678  0.         -0.01104466]
 [-0.01793641 -0.01104466  0.        ]]
